In [57]:
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import numpy as np
import copy

import matplotlib.pyplot as plt
import numpy as np
import tqdm
from sklearn.metrics import roc_curve
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
import math

In [113]:
def sequential_train_test_split(data, ratio = 0.8):
    #data.fillna(method='ffill', inplace=True)  # Forward fill
    #data.fillna(method='bfill', inplace=True)  # Backward fill
    #data.interpolate(method='linear', inplace=True)

    df = pd.DataFrame(data.dropna()).to_numpy()
    split_index = int(len(df) * ratio)
    return df[:split_index], df[split_index:]

def benchmark_accuracy(prediction, test):
    prediction = pd.DataFrame(prediction)
    test = pd.DataFrame(test)

    
    return (prediction == test).sum() / len(prediction)

def calculate_return(prediction, data) :
    data = data[:,0]
    prediction = pd.DataFrame(prediction)
    temp = pd.DataFrame(data)
    temp['log_return'] = np.log(temp / temp.shift(1))
    temp['returne'] = prediction.shift(1) * temp['log_return'].values.reshape(-1, 1)
    temp.dropna(inplace = True) 

    return np.exp(temp[['log_return', 'returne']].sum())

In [121]:
file = 'apple_output.csv'
stock_name = 'AAPL.O'

data = pd.read_csv(file, index_col = 0, parse_dates = True).dropna()
data['lag_1'] = data[stock_name].shift(1)
data['lag_2'] = data[stock_name].shift(2)
data['lag_3'] = data[stock_name].shift(3)
data['SMA_EMA_ratio'] = data['SMA'] / data['EMA']
SMA_short = 20
SMA_long = 50
data['SMA_Short'] = data[stock_name].rolling(SMA_short).mean()
data['SMA_Long'] = data[stock_name].rolling(SMA_long).mean()
data['SMA_Long_Short'] = data['SMA_Short'] / data['SMA_Long']
data['direction'] = np.where(data[stock_name] <= data[stock_name].shift(-1), 1, -1)

print(data.shape)

(1887, 20)


In [122]:
train_data, test_data = sequential_train_test_split(data)
print(train_data.shape, test_data.shape)
print(type(train_data), type(test_data))
X_train, y_train, X_test, y_test = train_data[:, :-1], train_data[:, -1], test_data[:, :-1], test_data[:, -1]
print(X_test.shape)
print(X_train.shape)


(1470, 20) (368, 20)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(368, 19)
(1470, 19)


In [129]:

max = 0
maxi = 0
maxret = 0
maxreti =0
log_ret = 0
max_i = 0
max_j = 0
max_k = 0
max_z = 0
max_a = 0
max_b = 0


for i in range(1, 100):
    print(i)
    for j in ('uniform', 'distance'):
        for k in ('auto', 'ball_tree', 'kd_tree', 'brute'):
            for z in range(1,50): 
                for a in (1, 2):    
                    for b in ('precomputed', 'minkowski'):
                        knn = KNeighborsClassifier(n_neighbors=i,  weights=j, algorithm=k, leaf_size=z, p=a, metric=b, n_jobs=2)


                        knn.fit(X_train, y_train)

                        prediction = knn.predict(X_test)

                        temp = benchmark_accuracy(prediction, y_test)
                        if (temp.values[0] > max) :
                            max = temp.values[0]
                            maxi = i
                        log, returne = calculate_return(prediction, X_test)
                        
                        if returne > maxret :
                            maxret = returne
                            maxreti = i
                            log_ret = log
                            max_j = j
                            max_k = k
                            max_z = z
                            max_a = a
                            max_b = b
    # if(tempi)
print(maxret, maxreti, log_ret, max_i,max_j,max_k,max_z,max_a,max_b )
print(max, maxi)

1
2
3
4
5
6
7


KeyboardInterrupt: 

In [130]:

print(maxret, maxreti, log_ret, max_i,max_j,max_k,max_z,max_a,max_b )
print(max, maxi)

1.786545044136534 5 1.6603282805632786 0 distance auto 1 2 precomputed
0.5380434782608695 5


In [8]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.metrics import mean_squared_error
import numpy as np
# Load your stock price dataset (replace 'your_dataset.csv' with your actual file path)
data = pd.read_csv('apple_output.csv', index_col = 0, parse_dates = True).dropna()
data['direction'] = np.where(data['AAPL.O'] <= data['AAPL.O'].shift(-1), 1, -1)

# Preprocess the data
X = data[['AAPL.O', 'SlowK', 'SlowD', 'RSI', 'ADX', 'CCI', 'Aroon Down',
       'Aroon Up', 'OBV', 'Chaikin A/D', 'SMA', 'EMA']]  # Replace 'feature1', 'feature2', ... with actual feature names
y = data['direction']  # Replace 'target_column' with the name of your target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
# #scaler = MinMaxScaler()
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.fit_transform(X_test)
scaler = StandardScaler()
X_train_scaled = X_train
X_test_scaled = X_test

# Create and train the KNN model
k = 5  # Number of neighbors
for i in range (1, 100):
       knn_model = KNeighborsClassifier(n_neighbors=i)
       knn_model.fit(X_train_scaled, y_train)

       # Make predictions
       y_pred = knn_model.predict(X_test_scaled)

       # Evaluate the model's performance
       mse = mean_squared_error(y_test, y_pred)
       print(f"Mean Squared Error: {mse}")

       print(benchmark_accuracy(y_test.values, y_pred))


Mean Squared Error: 2.0
0    0.5
dtype: float64
Mean Squared Error: 1.9682539682539681
0    0.507937
dtype: float64
Mean Squared Error: 1.9894179894179893
0    0.502646
dtype: float64
Mean Squared Error: 2.0634920634920637
0    0.484127
dtype: float64
Mean Squared Error: 2.052910052910053
0    0.486772
dtype: float64
Mean Squared Error: 2.1164021164021163
0    0.470899
dtype: float64
Mean Squared Error: 2.0211640211640214
0    0.494709
dtype: float64
Mean Squared Error: 1.9365079365079365
0    0.515873
dtype: float64
Mean Squared Error: 2.0423280423280423
0    0.489418
dtype: float64
Mean Squared Error: 2.0105820105820107
0    0.497354
dtype: float64
Mean Squared Error: 2.0105820105820107
0    0.497354
dtype: float64
Mean Squared Error: 1.9788359788359788
0    0.505291
dtype: float64
Mean Squared Error: 2.0211640211640214
0    0.494709
dtype: float64
Mean Squared Error: 2.0105820105820107
0    0.497354
dtype: float64
Mean Squared Error: 2.1164021164021163
0    0.470899
dtype: float64
M

In [ ]:
# Code Snippet: Fetching and Preparing Stock Data
import yfinance as yf
import numpy as np
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.preprocessing import StandardScaler

# Fetch historical stock data using Yahoo Finance API
stock_symbol = ‘AAPL’ # Replace with your desired stock symbol
stock_data = yf.download(stock_symbol, start=’2022–01–01', end=’2023–12–21')

# Feature engineering — assuming ‘Close’ is the target variable and other columns are features
features = [‘Open’, ‘High’, ‘Low’, ‘Volume’]
target = ‘Close’

# Splitting the data into features and target variable
X = stock_data[features]
y = stock_data[target]

# Convert data to NumPy arrays for scaling
X_np = X.values
y_np = y.values

# Code Snippet: Feature Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_np)
# Code Snippet: Random Forest Model Creation and Training
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_scaled, y_np)
#gimana kalo make and of a few predictions